# **Simulation in closed-loop**

In [33]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors

import package_LAB
from importlib import reload
package_LAB = reload(package_LAB)

from package_LAB import LL_RT, PID_RT, IMCTuning, Margins, Controller
from package_DBR import SelectPath_RT, Delay_RT, FO_RT, Process


In [34]:
#plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interactive, VBox

## Simulation parameters

In [35]:
TSim = 2000
Ts = 1
N = int(TSim/Ts) + 1 

## Dynamics and controller parameters

In [36]:
#DV 
Kp_ODV_SOPDT = 0.2951290424136788
T1_ODV_SOPDT = 182.2549613489765
T2_ODV_SOPDT = 13.184430234847984
theta_ODV_SOPDT = 28.999891911961512

#MV
Kp_OMV_SOPDT = 0.30788564834253684
T1_OMV_SOPDT = 183.81942938046797
T2_OMV_SOPDT = 3.2920224028341535e-12
theta_OMV_SOPDT = 20.015407110302775

#Operating points 
DV0 = 50 
MV0 = 50
PV0 = 49.3

# Set maximum and minimum MV values
MVmin = 0
MVmax = 100

# Coefficients
alpha = 0.7
gamma = 0.5

## Scenarios

In [37]:
scenario = 7

# 1 : Response to setpoint change
if scenario == 1:
    SPPath = {0: PV0 - 5, 1000: PV0 + 5}
    ManPath = {0: True, 500: False}
    MVManPath = {0: MV0}
    DVPath = {0: DV0}
    FF = False
    ManFF = True
# 2 : Response to perturbation, with controller in manual mode, without FF
elif scenario == 2:
    SPPath = {0: PV0}
    ManPath = {0: True}
    MVManPath = {0: MV0}
    DVPath = {0: DV0, 1500: DV0 + 10}
    FF = False
    ManFF = True
# 3 : Response to perturbation, with controller in manual mode, with FF
elif scenario == 3:
    SPPath = {0: PV0}
    ManPath = {0: True}
    MVManPath = {0: MV0}
    DVPath = {0: DV0, 1500: DV0 + 10}
    FF = True
    ManFF = True
# 4 : Response to perturbation, with controller in auto mode, without FF
elif scenario == 4:
    SPPath = {0: PV0}
    ManPath = {0: False}
    MVManPath = {0: MV0}
    DVPath = {0: DV0, 1500: DV0 + 10}
    FF = False
    ManFF = False # Not needed
# 5 : Response to perturbation, with controller in auto mode, with FF
elif scenario == 5:
    SPPath = {0: PV0}
    ManPath = {0: False}
    MVManPath = {0: MV0}
    DVPath = {0: DV0, 1500: DV0 + 10}
    FF = True
    ManFF = False # Not needed
# 6 : Response to perturbation & setpoint change, with controller in auto mode, with FF
elif scenario == 6:
    SPPath = {0: PV0 - 5, 1000: PV0 + 5}
    ManPath = {0: False}
    MVManPath = {0: MV0}
    DVPath = {0: DV0, 1500: DV0 + 10}
    FF = True
    ManFF = False # Not needed
# 7 : Response to perturbation & setpoint change, with controller in auto mode, with FF
elif scenario == 7:
    SPPath = {0: PV0 + 5, 1000: PV0 +10}
    ManPath = {0: True, 500: False, TSim: False}
    MVManPath = {0: MV0+15, TSim: MV0+15}
    DVPath = {0: DV0, 1600: DV0 + 10}
    FF = True
    ManFF = False # Not needed

## Simulation

In [38]:
# IMC tuning
Kc, Ti, Td = IMCTuning(Kp_OMV_SOPDT, T1_OMV_SOPDT, T2_OMV_SOPDT, theta_OMV_SOPDT, gamma, model="SOPDT")
print(f"Kc: {Kc}, Ti: {Ti}, Td: {Td}")

Kc: 5.33426261068635, Ti: 183.81942938047126, Td: 3.2920224028340946e-12


In [39]:
# Running simulation with chosen scenario

t = []

SP = []
PV = []
MAN = []
MV_MAN = []
DV = []
MVFF = []
MV = []
MVp = []
MVi = []
MVd = []
E = []
PV_p = []
PV_d = []

MVFF_Delay = []
MVFF_LL1 = []
MV_Delay_P = []
MV_FO_P = []
MV_Delay_D = []
MV_FO_D = []



for i in range(0, N):
    t.append(i * Ts)
    SelectPath_RT(SPPath, t, SP)
    SelectPath_RT(ManPath, t, MAN)
    SelectPath_RT(MVManPath, t, MV_MAN)
    SelectPath_RT(DVPath, t, DV)
    
    # FeedForward
    Delay_RT(DV - DV0*np.ones_like(DV), max(theta_ODV_SOPDT-theta_OMV_SOPDT, 0), Ts, MVFF_Delay)
    LL_RT(MVFF_Delay, -Kp_ODV_SOPDT/Kp_OMV_SOPDT, T1_OMV_SOPDT, T1_ODV_SOPDT, Ts, MVFF_LL1)
    if FF == True:
        LL_RT(MVFF_LL1, 1, T2_OMV_SOPDT, T2_ODV_SOPDT, Ts, MVFF)
    else:
        LL_RT(MVFF_LL1, 0, T2_OMV_SOPDT, T2_ODV_SOPDT, Ts, MVFF) # Set MVFF to 0 if FF is disabled
    
    # PID
    PID_RT(SP, PV, MAN, MV_MAN, MVFF, Kc, Ti, Td, alpha, Ts, MVmin, MVmax, MV, MVp, MVi, MVd, E, ManFF, PV0)
    
    # Process
    Delay_RT(MV, theta_OMV_SOPDT, Ts, MV_Delay_P, MV0)
    FO_RT(MV_Delay_P, Kp_OMV_SOPDT, T1_OMV_SOPDT, Ts, MV_FO_P)
    FO_RT(MV_FO_P, 1, T2_OMV_SOPDT, Ts, PV_p)
    
    # Disturbance
    Delay_RT(DV - DV0*np.ones_like(DV), theta_ODV_SOPDT, Ts, MV_Delay_D)
    FO_RT(MV_Delay_D, Kp_ODV_SOPDT, T1_ODV_SOPDT, Ts, MV_FO_D)
    FO_RT(MV_FO_D, 1, T2_ODV_SOPDT, Ts, PV_d)
    
    PV.append(PV_p[-1] + PV_d[-1] + PV0 - Kp_OMV_SOPDT*MV0)

In [40]:
#Plot
# Create figure
fig = go.FigureWidget(make_subplots(rows=4, cols=1, specs = [[{}], [{}], [{}], [{}]], vertical_spacing = 0.15, row_heights=[0.2, 1, 1, 0.2], subplot_titles=("Manual Mode", "MV and Components", "PV, SP and E", "Perturbation")))
fig.add_trace(go.Scatter(x=t, y=SP, name="SP"), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=PV, name="PV"), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=DV, name="DV"), row=4, col=1)
#fig.add_trace(go.Scatter(x=t, y=E, name="E", line=dict(dash='dash')), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=MV, name="MV"), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MVp, name="MVp", line=dict(dash='dash')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MVi, name="MVi", line=dict(dash='dash')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MVd, name="MVd", line=dict(dash='dash')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MAN, name="Man"), row=1, col=1)

# Update layout
fig['layout'].update(height=800, width=800)
fig['layout']['xaxis1'].update(title='Time (s)')
fig['layout']['yaxis3'].update(title='(°C)')
fig['layout']['xaxis2'].update(title='Time (s)')
fig['layout']['yaxis2'].update(title='MV (%)')
fig['layout']['xaxis3'].update(title='Time (s)')

l1, = ax1.step([0, TSim], [0, 1], 'k-', label="Man", where='post')
if scenario == 1 or scenario == 4 or scenario == 5:
    ax1.set_title('Closed-loop response with PID controller and no feedforward')
elif scenario == 2:
    ax1.set_title('Open-loop response with PID controller and no feedforward')
elif scenario == 3:
    ax1.set_title('Open-loop response with PID controller and feedforward')
elif scenario == 6 or scenario == 7:
    ax1.set_title('Closed-loop response with PID controller and feedforward')
ax1.legend(loc='best')

l2, = ax2.step([0,TSim],[0,100],'b-',linewidth=2,label="MV",where='post')
l3, = ax2.step([0,TSim],[0,100],'m--',linewidth=1,label="MVP",where='post')
l4, = ax2.step([0,TSim],[0,100],'c--',linewidth=1,label="MVI",where='post')
l5, = ax2.step([0,TSim],[0,100],'b--',linewidth=1,label="MVD",where='post')
l6, = ax2.step([0,TSim],[0,100],'r--',linewidth=1,label="MVFF",where='post')
ax2.set_ylabel("Value of MV [%]")
ax2.legend(loc='best')
ax2.grid()

l7, = ax3.step([0,TSim],[0,100],'k-',linewidth=2,label="SP",where='post')
l8, = ax3.step([0,TSim],[0,100],'g-',linewidth=1,label="PV",where='post')
ax3.set_ylabel("Value of PV [°C]")
ax3.legend(loc='best')
ax3.grid()

l9, = ax4.step([0,TSim],[0,100],'r-',linewidth=2,label="DV",where='post')
ax4.set_xlabel("Time [s]")
ax4.set_ylabel("Value of DV [%]")
ax4.legend(loc='best')

# Set Data
l1.set_data(t,MAN)
l2.set_data(t,MV)
l3.set_data(t,MVp)
l4.set_data(t,MVi)
l5.set_data(t,MVd)
l6.set_data(t,MVFF)
l7.set_data(t,SP)
l8.set_data(t,PV)
l9.set_data(t,DV)

ax1.set_xlim(0, t[-1])
ax2.set_xlim(0, t[-1])
ax3.set_xlim(0, t[-1])
ax4.set_xlim(0, t[-1])

MVMinLimit = myRound(np.min((np.min(MV),np.min(MVp),np.min(MVi),np.min(MVd),np.min(MVFF))),5)
MVMaxLimit = myRound(np.max((np.max(MV),np.max(MVp),np.max(MVi),np.max(MVd),np.max(MVFF))),5)

ax1.set_ylim(-0.1,1.1)
ax2.set_ylim(MVMinLimit-10, MVMaxLimit+10)
ax3.set_ylim(myRound(np.min((np.min(PV),np.min(SP))),5)-5, myRound(np.max((np.max(PV),np.max(SP))),5)+5)
ax4.set_ylim(myRound(np.min(DV),5)-5, myRound(np.max(DV),5)+5)

# Save plot
# if not os.path.exists('Plots'):
#     os.makedirs('Plots')

# plt.savefig(f'Plots/Simulation_scenario_{scenario}.png',transparent=True)
# plt.savefig(f'Plots/Simulation_scenario_{scenario}.pdf',transparent=True)

# Save data
# t = np.array(t)
# MAN = np.array(MAN)
# MV = np.array(MV)
# MVp = np.array(MVp)
# MVi = np.array(MVi)
# MVd = np.array(MVd)
# MVFF = np.array(MVFF)
# SP = np.array(SP)
# PV = np.array(PV)
# DV = np.array(DV)

# data = np.vstack((t.T, MAN.T, MV.T, MVp.T, MVi.T, MVd.T, MVFF.T, SP.T, PV.T, DV.T))
# data = data.T
# if not os.path.exists('Data'):
#     os.makedirs('Data')
# nameFile = f'Data/Experiment_scenario_{scenario}.txt'
# np.savetxt(nameFile,data,delimiter=',',header='t,MAN,MV,MVp,MVi,MVd,MVFF,SP,PV,DV',comments='')

FigureWidget({
    'data': [{'name': 'SP',
              'type': 'scatter',
              'uid': 'e66bee68-d113-41e7-a546-c2b9f336debb',
              'x': [0, 1, 2, ..., 1998, 1999, 2000],
              'xaxis': 'x3',
              'y': [54.3, 54.3, 54.3, ..., 59.3, 59.3, 59.3],
              'yaxis': 'y3'},
             {'name': 'PV',
              'type': 'scatter',
              'uid': '4864ecb3-b107-4398-9575-32f07cb91243',
              'x': [0, 1, 2, ..., 1998, 1999, 2000],
              'xaxis': 'x3',
              'y': [33.90571758287315, 33.989011219633525, 34.07185418063439, ...,
                    59.297238280214984, 59.29724955319824, 59.297260807613156],
              'yaxis': 'y3'},
             {'name': 'DV',
              'type': 'scatter',
              'uid': 'c116a4c6-d962-4a63-8b35-0504e4d7cde0',
              'x': [0, 1, 2, ..., 1998, 1999, 2000],
              'xaxis': 'x4',
              'y': [50, 50, 50, ..., 60, 60, 60],
              'yaxis': 'y4'},
        